# Project 3: Communicating data findings


## 1. Dataset: Student Performance in Exams
This dataset is about student performance, where it lists the students' grades in 3 subjects, as well as information about the student, like their gender and ethnicity. 

The dataset contains 8 columns:
* `gender`: a categorical variable (male or female)
* `race/ethnicity`: a categorical variable, taking one of five values: `group A, group B...group E` 
* `parental level of education`: categorical variable, which can be `high school, college, bachelor's, master's or associate degress`
* `lunch`: a categorical variable, which tells the type of lunch the students have. This reflects their financial status. It can be `standard` or `free\reduced`
* `test preparation course`: indicates whether a student completed such course. The values can be either `completed` or `none`
* The rest of the columns are numeric, indicating the studen't's score, out of 100, in `math`, `reading` and `writing`

The dataset has **1000 records**, but we will see if that number changes after some wrangling

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
base_color = sb.color_palette()[0]
%matplotlib inline

import warnings
warnings.simplefilter("ignore")

In [ ]:
df = pd.read_csv("../input/students-performance-in-exams/StudentsPerformance.csv")
df.head()

## 2. Data wrangling
In this step, I will perform the following:
* checking the column names are "code friendly"
* checking for nulls in the data
* checking if there are duplicate rows
* checking if the datatypes are the most suitable for each column

This process is iterative, so I might need to do some data wrangling later

#### 2.1 Code friendly colum names
From the dataframe head, we can see that the column names are meaningful, but can sometimes be long or contain whitespace, which are not code friendly. As such, I will try to rename them

In [ ]:
# replace column names with values in the dictionary
df_mod = df.rename(columns={
    "race/ethnicity":"race",
    "parental level of education": "parent_education",
    "test preparation course": "prep_course"
})

# replace whitespace with underscore
df_mod.rename(columns= lambda x: x.replace(" ", "_"), inplace = True)

df_mod.head()

#### 2.2 Nulls, and datatypes
This can be easily checked by the `info()` method of the datafram

In [ ]:
df_mod.info()

So, 
* the data has no null values,
* The test scores are integers, and the categorical data is of type string

So far so good. **However**, for ordinal data, like parent_education. It will be useful to sort ordinal data while visualizing. We can do that via the `pd.api.CategoricalDatatype()` function to create an ordinal datatype, and convert the dataframe column to that datatype

###### Ordinal Datatype

In [ ]:
# create a list of sorted data
sorted_edu_levels = ["some high school", "high school", "some college", "associate's degree",
                     "bachelor's degree", "master's degree"]
# create the oridnal datatype
parent_education_ordinal = pd.api.types.CategoricalDtype(categories=sorted_edu_levels,
                                                        ordered = True)
# cast to this new type
df_mod["parent_education"] = df_mod["parent_education"].astype(parent_education_ordinal)

The rest of the ordinal columns, like lunch or prep course, are binary anyways, so we do not need to convert them

#### 2.3 Duplicated rows

In [ ]:
print("Are there any duplicated rows?")
df.duplicated().any()

That's it for data wrangling, unless something else comes up later

## 3. Exploratory data analysis

### 3.1 Univariate analysis
Let's see how each feature is distributed

#### 3.1.1 Gender

In [ ]:
sb.countplot(data = df_mod, x = "gender");
plt.title("Is the dataset biased towards one gender?");

The margin between the 2 gender count is not wide, sow we can assume they are equally represented.

#### 3.1.2 race

In [ ]:
# value_counts() for sorted bars (polished visual)
race_counts = df_mod.race.value_counts()
# unify the color
base_color = sb.color_palette()[0]
# plot
sb.countplot(data = df_mod, x = "race", color = base_color, order=race_counts.index);
plt.title("How are differnet races distributed?");

So, different race groups are not equally represented. We will need to keep that in mind if we will take into consideration the race/ethinicity in further analysis

#### 3.1.3 Parents level of education

In [ ]:
# value_counts() for sorted bars (polished visual)
edu_level_count = df_mod.parent_education.value_counts()
# unify the color
base_color = sb.color_palette()[0]
# plot
sb.countplot(data = df_mod, y = "parent_education", color = base_color);
plt.title("What's the educational level of parents?");
plt.ylabel("Parental level of education", ha = "right", rotation = 0, color = "blue");
plt.xlabel("Count", color = "blue");

We can see that the majority of parents have associate's degrees or less, while fewer have bachelor's degree, and even fewer have a master's degree

#### 3.1.4 Lunch

In [ ]:
sb.countplot(data = df_mod, x = "lunch");
plt.title("What type of lunch do students have?");

We can see that the majority of students enjoy `standard` lunch, wheras the rest have `free` lunch, which may indicate a poor financial situation.

#### 3.1.5 Taking Test Preparation Course

In [ ]:
sb.countplot(data = df_mod, x = "prep_course");
plt.title("Did students complete a test preparation course?");
plt.xlabel("");
plt.xticks(ticks = [0, 1], labels=["Didn't complete a course","Completed a course"]);

As we can see, the majority of students didn't complete a test preparation course

#### 3,1.6 Math, reading and writing scores
Becuase score is an integer between [0 and 100], we can plot a histogram with binsize = 1. Or, to make the trend more visible, we can group the score into bins of 5

In [ ]:
# bins of size 5 and include the range [0 -> 100]
bins = np.arange(start = 0, stop = 101, step = 4) - 0.1

plt.figure(figsize= (15,5));
# plot math score
plt.subplot(1,3,1);
plt.hist(data = df_mod, x = "math_score", bins = bins);
plt.xlabel("Math Score");
plt.ylabel("Count");
plt.title("Histogram of Math Score");

# plot reading score
plt.subplot(1,3,2);
plt.hist(data = df_mod, x = "reading_score", bins = bins);
plt.xlabel("Reading Score");

plt.ylabel("Count");
plt.title("Histogram of Reading Score");

# plot the writing score
plt.subplot(1,3,3);
plt.hist(data = df_mod, x = "writing_score", bins = bins);
plt.xlabel("Writing Score");
plt.ylabel("Count");
plt.title("Histogram of Writing Score");

<br>
We can see that the scores resemble a Gaussian distribution, and skewed to the left, where the average grade lies in the range [60 to 80]

#### This concludes our univariate analysis

### 3.2 Bivariate Analysis

#### 3.2.1 Score Redundant information?
It is obvious that any further analysis will be connected with the test scores. However, instead of analysing the effect of each factor with each score, we can summarize the test scores in 1 variable, if the scores are highly correlated **(feature engineering)**

Let's see how test scores are related

###### First, let's see the distributions overlapped in one plot

In [ ]:
# allocate appropriate bin sizes
bins = np.arange(start = 0, stop = 101, step = 4) - 0.1

# plot, adjusting the transparency
plt.hist(data = df_mod, x = "math_score", bins = bins, alpha = 0.3);
plt.hist(data = df_mod, x = "reading_score", bins = bins, alpha = 0.3);
plt.hist(data = df_mod, x = "writing_score", bins = bins, alpha = 0.3);
plt.title("All Test score distribution");
plt.legend(["math", "reading", "writing"], loc = "upper left");
plt.xlabel("Score");
plt.ylabel("Count");

* So, we can see that the distributions are very overlapping. However, this doesn't gurantee that those who did well in a subject did well in the rest. It could be the case that some students perform well in `maths` and not so well in `reading`, (and vice versa) and we will still have the same overlapping histograms. 

* For that, we need another plot, like scatter plots

###### Scatter plots of test scores

In [ ]:
plt.figure(figsize=(15,5));
alpha = 0.3

# math score vs reading score
plt.subplot(1,3,1);
plt.scatter(y = df_mod.math_score, x = df_mod.reading_score, alpha = alpha);
plt.xlabel("Reading score");
plt.ylabel("Math score");
plt.title("Math score vs Reading score");

# math score vs writing score
plt.subplot(1,3,2);
plt.scatter(y = df_mod.math_score, x = df_mod.writing_score, alpha = alpha);
plt.xlabel("Writing score");
plt.ylabel("Writing score");
plt.title("Math score vs Writing score");

# writing score vs reading score
plt.subplot(1,3,3);
plt.scatter(y = df_mod.writing_score, x = df_mod.reading_score, alpha = alpha);
plt.xlabel("Writing score");
plt.ylabel("Reading score");
plt.title("Writing score vs Reading score");

* We can see that all test scores are **highly corrlated**, with reading and writing scores being the most correlated.
* We can use this observation to include a new feature, and eliminate some redundancy in the data
* We can create a new feature `avg_score` to be the average of the three scores
* However, because `reading` and `writing` are more correlated, we can make their weights less

In [ ]:
# create new feature (weighted average of all scores)
df_mod["avg_score"] = 0.5 * df_mod.math_score + 0.25 * df_mod.writing_score + 0.25 * df_mod.reading_score

# drop the scores features
df_mod.drop(columns = ["math_score", "reading_score", "writing_score"], inplace = True)

# display the head
df_mod.head()

#### 3.2.2 What is the performance per gender?

In [ ]:
sb.boxplot(data = df_mod, x = "gender", y = "avg_score");

As we can see, the performace of females is slightly better, but with more outliers in the lower scores. The margin between the groups performances is too small however.

#### 3.2.3 What is the performance of different racial groups?

In [ ]:
# find the order
order = df_mod.groupby("race").median().iloc[::1].index

# plot
sb.boxplot(data = df_mod, x = "race", y = "avg_score", color=base_color, order = order);
plt.ylabel("Average Score");
plt.title("Average score vs Racial groups");

So, it looks that racial `group E` has the best performance while racial `group A` has the worst. We will try to see whether other factors influence this result in multivariate analysis

#### 3.2.4 Does the type of lunch affect student performance?

In [ ]:
sb.boxplot(data = df_mod, x = "lunch", y = "avg_score");
plt.ylabel("Average Score");
plt.title("Average score vs Racial groups");

As we can see, the score differs with the type of lunch the students have, with poeple having `reduced` lunch scoring lower. This can also reflect financial status of these student's families

#### 3.2.5 What is relation between the parents' level of education and test scores?

In [ ]:
sb.barplot(data = df_mod, y = "parent_education", x = "avg_score", color = base_color);
plt.ylabel("Parents' educational level", rotation = 0, ha = "right", color = "Blue");
plt.xlabel("Average Score", color = "Blue");
plt.title("Average score for different educational level of parents");

It looks like the higher the educational level of the parent, the better the grade

#### 3.2.6 What is the relation between completing a preparation course, and the average score?

In [ ]:
sb.boxplot(data = df_mod, x = "prep_course", y = "avg_score");
plt.ylabel("Average Score");
plt.xlabel("Preparation Course");
plt.title("Average Score vs taking a preparation course");

As one might expect, people completing a preparation course tend to fare better in exams

### 3.3 So far, we have done bivariate analysis on every variable with the average score, and reached the following observations:
* Both genders display nearly similar performance
* Racial group E is the best performing group, whereas racial group B is the worst performing group
* Students that completed a preparation course tend to perfrom better
* Students with standard lunch meals tend to perform better than students with reduced lunch
* Students scores tend to improve with higher educaitonal levels from their parents

I will try to explore further the relation betwee racial group and other elements like lunch type, parental educational levels, and preparation course, in order to investigate why some racial groups perform better 

#### 3.3.1 If lunch reflects financial status, is there a correlation with not being able to pay for lunch, and not being able to pay for a preparation course?

In [ ]:
sb.countplot(data = df_mod, x = "lunch", hue = "prep_course");

* It looks like course completion is nearly the same whethee students take standard lunch or not
* We can visualize better if we plot the ratio of course completion per lunch type

In [ ]:
df_mod.groupby("lunch")["prep_course"].value_counts()

In [ ]:
# groupby lunch type, and get the value counts
grouped = df_mod.groupby("lunch")["prep_course"].value_counts()

# get the ratios
comp_ratio_reduced =  grouped["free/reduced"]["completed"] / grouped["free/reduced"]["none"]
comp_ratio_standard = grouped["standard"]["completed"] / grouped["standard"]["none"]

# plot
sb.barplot(x = ["Standard Lunch", "Reduced Lunch"], 
        y = [comp_ratio_standard,comp_ratio_reduced ]);
plt.title("Ratio of Preparation course completion vs Lunch type");
plt.xlabel("Type of lunch");
plt.ylabel("Completion ratio");

So, it looks like there is not much difference in course completion rate between those who take standard meals and others

#### 3.3.2 What kind of lunch do different racial groups have?

In [ ]:
df_mod.groupby("race")["lunch"].value_counts()

In [ ]:
# groupby lunch type, and get the value counts (it's faster to use lambdas)
grouped_race_lunch = df_mod.groupby("race")["lunch"].agg(lambda x: x.value_counts()["standard"]/x.value_counts()["free/reduced"])
grouped_race_lunch

In [ ]:
# get the order
ordered = grouped_race_lunch.sort_values()

# plot
sb.barplot(x = ordered.index, y = ordered.values, color = base_color);
plt.title("How well racial groups have lunch");
plt.ylabel("Standard to reduced lunch ratio", ha = "right", rotation = 0);

So, it looks like group E enjoys the best standard to reduced lunch ratio, whereas group A has the worst. This is consistent with
* The direct relationship between having standard lunch and good score
* Group E has the best average score, whereas group A has the worst score

#### 3.3.3 How do racial groups differ with completing a preparation course?

In [ ]:
# groupby lunch type, and get the value counts (it's faster to use lambdas)
grouped_race_prep = df_mod.groupby("race")["prep_course"].agg(lambda x: x.value_counts()["completed"]/x.value_counts()["none"])
grouped_race_prep
ordered = grouped_race_prep.sort_values()

# plot
sb.barplot(x = ordered.index, y = ordered.values, color = base_color);
plt.ylabel("Preparation course completion ratio", rotation = 0, ha = "right");
plt.title("How different ratial groups complete the preparation course");

As we can see, group A is the second least group in completing the preparation course, whereas group E is the highest. This is consistent with our observations that
* Preparation course completion tends to improve grads
* group E is the best performing group, and the best in course completion ratio. On the opposite side is group A

#### 3.3.4 Now let's see the proportion of parental level of education for each racial group
Because racial groups have different numbers of records, we should opt to the proportion, and not the absolute count. For that, I will
* Groupby race
* For each race group, calculate the proportion of each educational level

In [ ]:
# the lambda function calculates the proportions
race_grouped = df_mod.groupby("race")["parent_education"].apply(lambda x: x.value_counts()/x.count())
race_grouped.name = "proportion"

# convert to dataframe
race_grouped = race_grouped.to_frame().reset_index().rename(columns = {"level_1": "parent_education"})
race_grouped.head()

In [ ]:
# now we plot
plt.figure(figsize=(10,7));
sb.barplot(data = race_grouped,y = "proportion", hue = "parent_education",x = "race");
plt.xticks(rotation = 20);
plt.xlabel("Parents' level of education");
plt.title("What's the proportion of educational levels for different racial groups?");
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

We can see that 
* For group A, the proportions are skewed towards lower educational levels
* Then we progress to group E, we find that the educational levels proportions resembel a "Gaussian", with the majority of educational levels in the college and associate's degrees

We can argue that this also is consistent with our previous findings, where
* Students whose parents have higher educational levels tend to score more
* group E has the highest score, and the most balanced among parent's educational levels
* group A is has the lowest score, and are skewed towards lower educational levels for parents.


#### As such, from points 3.3.2, upto 3.3.4,  we can also hypothesize that the bad performance of other racial groups can be influended by:
* Parent's level of education
* Whether or not students have completed a preparatory course
* Whether or not students enjoy a standard meal or a reduced meal

That is the reason why different racial groups have different scores can be due to other factors rather than race itself. Therefore I suggest that further investigation is required, and facilities be provided for those in need, in order to offer equal opportunities to students of different racial groups.

### 3.4 Multivariate analysis
We can now visualize our major bivaraite outcomes using a Faceted Plot. 
* I will make the rows refer to the `lunch` attribute and the rows refer to the `preparation course` attribute. 
* Inisde the axes, I will use a barplot to view the `Average Score` vs `Parent's educational level` 

In [ ]:
g = sb.FacetGrid(data = df_mod, row="lunch", col = "prep_course", height = 3,aspect=2,  margin_titles=True);
g.map(sb.barplot, "avg_score", "parent_education");
g.set_axis_labels("Averag Score");

From the plot, we can see that:
* In all 4 figures, there is a trend where the students' score increases as the parents' level of educaiton increases
* If we move from the figures in the left to the right (From not completing a preparation course to completing one), the average score increases.
* If we move from the figures in the bottom to the top (From reduced meal to standard meal), the average score increases.